In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType

spark = SparkSession.builder.appName("Jupyter")\
    .getOrCreate()



25/08/04 18:14:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [63]:
spark.stop()

In [2]:
events_schema = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("device_id", IntegerType(), False),
    StructField("referrer", StringType()),
    StructField("host", StringType(), False),
    StructField("url", StringType(), False),
    StructField("event_time", TimestampType(), False)
])
devices_schema = StructType([
    StructField("device_id", IntegerType(), False),
    StructField("browser_type", StringType()),
    StructField("os_type", StringType(), False),
    StructField("device_type", StringType(), False)
])

events = spark.read.option("header", "true")\
    .csv("/home/iceberg/data/events.csv", events_schema)\
    .withColumn("event_date", expr("DATE(DATE_TRUNC('day', event_time))"))
devices = spark.read.option("header","true")\
    .csv("/home/iceberg/data/devices.csv", devices_schema)

df = events.join(devices,on="device_id",how="left")
df = df.withColumnsRenamed({'browser_type': 'browser_family', 'os_type': 'os_family'})

df.show()

+----------+-----------+--------+--------------------+----------+--------------------+----------+--------------+---------+-----------+
| device_id|    user_id|referrer|                host|       url|          event_time|event_date|browser_family|os_family|device_type|
+----------+-----------+--------+--------------------+----------+--------------------+----------+--------------+---------+-----------+
| 532630305| 1037710827|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08|         Other|    Other|      Other|
| 532630305|  925588856|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10|         Other|    Other|      Other|
| 532630305|-1180485268|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17|         Other|    Other|      Other|
| 532630305|-1044833855|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24|         Other|    Other|      Other|
| 532630305|  747494706|    NULL| www.zachwilson.tech| 

In [ ]:
df.schema

In [3]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.show()
sortedTwo.show()


+-----------+-----------+--------+--------------------+------------+--------------------+----------+--------------+---------+------------------+
|  device_id|    user_id|referrer|                host|         url|          event_time|event_date|browser_family|os_family|       device_type|
+-----------+-----------+--------+--------------------+------------+--------------------+----------+--------------+---------+------------------+
| -643696601| 1272828233|    NULL|admin.zachwilson....|           /|2021-01-17 02:54:...|2021-01-17|        Chrome|  Windows|             Other|
|  532630305|  747494706|    NULL|admin.zachwilson....|           /|2021-01-17 15:41:...|2021-01-17|         Other|    Other|             Other|
|  532630305|  683643637|    NULL|admin.zachwilson....|           /|2021-01-17 19:04:...|2021-01-17|         Other|    Other|             Other|
|  532630305|  683643637|    NULL|    www.eczachly.com|           /|2021-01-17 14:34:...|2021-01-17|         Other|    Other|     

In [ ]:
# .sortWithinPartitions() sorts within partitions, whereas .sort() is a global sort, which is very slow

# Note - exchange is synonymous with Shuffle

In [4]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.explain()
sortedTwo.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [event_date#12 ASC NULLS FIRST, host#3 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(event_date#12, 10), REPARTITION_BY_NUM, [plan_id=367]
      +- Project [device_id#1, user_id#0, referrer#2, host#3, url#4, event_time#5, event_date#12, browser_type#22 AS browser_family#40, os_type#23 AS os_family#41, device_type#24]
         +- BroadcastHashJoin [device_id#1], [device_id#21], LeftOuter, BuildRight, false
            :- Project [user_id#0, device_id#1, referrer#2, host#3, url#4, event_time#5, cast(date_trunc(day, event_time#5, Some(Etc/UTC)) as date) AS event_date#12]
            :  +- FileScan csv [user_id#0,device_id#1,referrer#2,host#3,url#4,event_time#5] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:int,device_id:int,referrer:string,host:string,url:string,event_time:

In [5]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|         demo|
|spark_catalog|
+-------------+



In [6]:
# spark.catalog.currentCatalog()
# spark.catalog.setCurrentCatalog('spark_catalog')
spark.conf.get('spark.sql.catalog.demo.s3.endpoint')


'http://minio:9000'

In [19]:
spark.sql("""
    CREATE DATABASE IF NOT EXISTS db
""")

DataFrame[]

In [7]:
spark.sql("""
    SHOW NAMESPACES
""").show()

#spark.sql("""
#    DROP NAMESPACE db
#""")

+---------+
|namespace|
+---------+
+---------+



In [11]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS bootcamp.events (
        url STRING,
        referrer STRING,
        browser_family STRING,
        os_family STRING,
        device_family STRING,
        host STRING,
        event_time TIMESTAMP,
        event_date DATE
    )
    USING iceberg
    PARTITIONED BY (year(event_time))
    TBLPROPERTIES ('write.distribution-mode'='hash')
""")

DataFrame[]

In [8]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
+---------+



In [9]:
%%sql

CREATE NAMESPACE IF NOT EXISTS bootcamp

++
||
++
++

In [10]:
%%sql

SHOW NAMESPACES

namespace
bootcamp


In [ ]:
%%sql

USE bootcamp

In [ ]:
%%sql

DROP NAMESPACE bootcamp

In [ ]:
%%sql

SHOW TABLES FROM bootcamp

In [42]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [43]:
%%sql

DROP TABLE IF EXISTS bootcamp.events_sorted

++
||
++
++

In [44]:
%%sql

DROP TABLE IF EXISTS bootcamp.events_unsorted

++
||
++
++

In [45]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    device_id INTEGER,
    user_id INTEGER, 
    referrer STRING,
    url STRING,
    browser_family STRING,
    os_family STRING,
    device_type STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (year(event_time))
TBLPROPERTIES ('write.distribution-mode'='hash');


++
||
++
++

In [12]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    device_id INTEGER,
    user_id INTEGER, 
    referrer STRING,
    url STRING,
    browser_family STRING,
    os_family STRING,
    device_type STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (year(event_time))
TBLPROPERTIES ('write.distribution-mode'='hash');

++
||
++
++

In [13]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    device_id INTEGER,
    user_id INTEGER, 
    referrer STRING,
    url STRING,
    browser_family STRING,
    os_family STRING,
    device_type STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (year(event_time))
TBLPROPERTIES ('write.distribution-mode'='hash');

++
||
++
++

In [14]:
%%sql

describe table bootcamp.events_unsorted

col_name,data_type,comment
device_id,int,None
user_id,int,None
referrer,string,None
url,string,None
browser_family,string,None
os_family,string,None
device_type,string,None
host,string,None
event_time,timestamp,None
event_date,date,None


In [15]:
spark.catalog.listTables('bootcamp')

[Table(name='events', catalog='demo', namespace=['bootcamp'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='events_sorted', catalog='demo', namespace=['bootcamp'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='events_unsorted', catalog='demo', namespace=['bootcamp'], description=None, tableType='MANAGED', isTemporary=False)]

In [16]:

start_df = df.repartition(4, col("event_time"))
    
first_sort_df = start_df.sortWithinPartitions(col("event_time"), col("host"))

start_df.writeTo("demo.bootcamp.events_unsorted").append()
first_sort_df.writeTo("demo.bootcamp.events_sorted").append()

In [17]:
start_df.show()

+----------+-----------+--------+--------------------+--------------------+--------------------+----------+-----------------+---------+-----------+
| device_id|    user_id|referrer|                host|                 url|          event_time|event_date|   browser_family|os_family|device_type|
+----------+-----------+--------+--------------------+--------------------+--------------------+----------+-----------------+---------+-----------+
| 532630305|-1044833855|    NULL| www.zachwilson.tech|                   /|2021-09-24 15:53:...|2021-09-24|            Other|    Other|      Other|
| 532630305| -824540328|    NULL|    www.eczachly.com|                   /|2021-09-29 01:22:...|2021-09-29|            Other|    Other|      Other|
|-906264142|-1913422462|    NULL|    www.eczachly.com|                   /|2021-02-23 00:17:...|2021-02-23|             curl|    Other|      Other|
| 532630305|  925588856|    NULL| www.zachwilson.tech|                   /|2021-05-10 18:12:...|2021-05-10|     

In [18]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM bootcamp.events_unsorted.files




size,num_files,sorted
5518734,3,sorted
5725128,3,unsorted


In [19]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
None,0


In [ ]:
%%sql

SHOW CURRENT NAMESPACE

In [20]:
%%sql 

SELECT *
FROM demo.bootcamp.events_unsorted.files

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://warehouse/bootcamp/events_unsorted/data/event_time_year=2023/00000-50-a2f0d574-a851-4bb7-a72a-d1c99876c810-0-00001.parquet,PARQUET,0,Row(event_time_year=53),292855,4314368,"{1: 368032, 2: 1002206, 3: 269274, 4: 823002, 5: 145585, 6: 120670, 7: 125151, 8: 76079, 9: 1350549, 10: 24095}","{1: 292855, 2: 292855, 3: 292855, 4: 292855, 5: 292855, 6: 292855, 7: 292855, 8: 292855, 9: 292855, 10: 292855}","{1: 0, 2: 67, 3: 128072, 4: 0, 5: 1, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0}",{},"{1: bytearray(b'\x7f\xba\x06\x80'), 2: bytearray(b'\xb1\xf4\x00\x80'), 3: bytearray(b'""https://www.goo'), 4: bytearray(b'""/?""""<?=print(93'), 5: bytearray(b'%E3%82%A6%E3%82%'), 6: bytearray(b'Android'), 7: bytearray(b'$2'), 8: bytearray(b'aashish.techcrea'), 9: bytearray(b'\x18\x8f~\x9c(\xf1\x05\x00'), 10: bytearray(b'\x9eK\x00\x00')}","{1: bytearray(b'm:\xff\x7f'), 2: bytearray(b'\xd6\xf4\xfe\x7f'), 3: bytearray(b'zachwilson.tech'), 4: bytearray(b'/zzageqnf.php?Fp'), 5: bytearray(b'webprosbot'), 6: bytearray(b'webOS'), 7: bytearray(b'vivo $2'), 8: bytearray(b'zzz.techcreator/'), 9: bytearray(b'HE\xdbM\xb3\x03\x06\x00'), 10: bytearray(b'\x89L\x00\x00')}",None,[4],None,0,None,None,None,"Row(browser_family=Row(column_size=145585, value_count=292855, null_value_count=1, nan_value_count=None, lower_bound='%E3%82%A6%E3%82%', upper_bound='webprosbot'), device_id=Row(column_size=368032, value_count=292855, null_value_count=0, nan_value_count=None, lower_bound=-2147042689, upper_bound=2147433069), device_type=Row(column_size=125151, value_count=292855, null_value_count=1, nan_value_count=None, lower_bound='$2', upper_bound='vivo $2'), event_date=Row(column_size=24095, value_count=292855, null_value_count=0, nan_value_count=None, lower_bound=datetime.date(2023, 1, 1), upper_bound=datetime.date(2023, 8, 24)), event_time=Row(column_size=1350549, value_count=292855, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2023, 1, 1, 0, 6, 50, 79000), upper_bound=datetime.datetime(2023, 8, 24, 23, 8, 20, 509000)), host=Row(column_size=76079, value_count=292855, null_value_count=0, nan_value_count=None, lower_bound='aashish.techcrea', upper_bound='zzz.techcreator/'), os_family=Row(column_size=120670, value_count=292855, null_value_count=1, nan_value_count=None, lower_bound='Android', upper_bound='webOS'), referrer=Row(column_size=269274, value_count=292855, null_value_count=128072, nan_value_count=None, lower_bound='""https://www.goo', upper_bound='zachwilson.tech'), url=Row(column_size=823002, value_count=292855, null_value_count=0, nan_value_count=None, lower_bound='""/?""""<?=print(93', upper_bound='/zzageqnf.php?Fp'), user_id=Row(column_size=1002206, value_count=292855, null_value_count=67, nan_value_count=None, lower_bound=-2147421007, upper_bound=2147415254))"
0,s3://warehouse/bootcamp/events_unsorted/data/event_time_year=2022/00001-51-a2f0d574-a851-4bb7-a72a-d1c99876c810-0-00001.parquet,PARQUET,0,Row(event_time_year=52),74035,920087,"{1: 90844, 2: 209411, 3: 44738, 4: 62169, 5: 45108, 6: 24719, 7: 24978, 8: 11527, 9: 386016, 10: 16402}","{1: 74035, 2: 74035, 3: 74035, 4: 74035, 5: 74035, 6: 74035, 7: 74035, 8: 74035, 9: 74035, 10: 74035}","{1: 0, 2: 0, 3: 43593, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}",{},"{1: bytearray(b'\xdf\n\xa5\x80'), 2: bytearray(b'\x993\x00\x80'), 3: bytearray(b'3.220.57.224'), 4: bytearray(b'/'), 5: bytearray(b') Bot'), 6: bytearray(b'Android'), 7: bytearray(b'17MB150WB'), 8: bytearray(b'admin.zachwilson'), 9: bytearray(b'\x88\xe5\x9fkz\xd4\x05\x00'), 10: bytearray(b'1J\x00\x00')}","{1: bytearray(b'\xb6!\xfd\x7f'), 2: bytearray(b'o\x96\xfd\x7f'), 3: bytearray(b'zachwilson.tech'), 4: bytearray(b'/zoushi'), 5: bytearray(b'webpr